In [10]:
# 수집한 콜금리를 한달 전 콜금리와 비교해 up, down, same 라벨링한다.
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [11]:
callrate = pd.read_csv('../data/call_rate.csv')
callrate

,Unnamed: 0,date,callrate
0,1,2011-12-31,3.29
1,2,2012-01-01,3.29
2,3,2012-01-02,3.25
3,4,2012-01-03,3.25
4,5,2012-01-04,3.24
...,...,...,...
3971,3972,2022-11-14,3.07
3972,3973,2022-11-15,3.03
3973,3974,2022-11-16,3.03
3974,3975,2022-11-17,3.02


In [12]:
# 1. 날짜를 기준으로 정렬하기
callrate = callrate.sort_values(by=['date'])

# 2. 비어있는 날짜(주말)와 금리를 추가하기
def all_date_range_df(date_range):

    df2 = pd.DataFrame({'date':date_range})
    
    return df2

date_range = (pd.date_range(start='20111231', end='20221118')).strftime("%Y-%m-%d").tolist()
df2 = all_date_range_df(date_range)

# 3. 두 df를 병합하고 결측값 채우기
def df_outer_join_merge(df1, df2):
    
    df_OUTER_JOIN = pd.merge(df1, df2, left_on='date', right_on='date', how='outer', sort=True)
    
    # 공휴일에 없는 환율정보는 전날을 기준으로 결측값 채우기
    df_finally = df_OUTER_JOIN.fillna(method='ffill')
    
    return df_finally

fin_df = df_outer_join_merge(callrate, df2)
fin_df = fin_df.set_index('date')
fin_df

,Unnamed: 0,callrate
date,,
2011-12-31,1,3.29
2012-01-01,2,3.29
2012-01-02,3,3.25
2012-01-03,4,3.25
2012-01-04,5,3.24
...,...,...
2022-11-14,3972,3.07
2022-11-15,3973,3.03
2022-11-16,3974,3.03


In [13]:
# 4. 한달 전 날짜 추가하기
month_date = []
for target_date in fin_df.index:
    
    month_before = (datetime.strptime(target_date,'%Y-%m-%d' ) - relativedelta(months = 1)).strftime('%Y-%m-%d')
    month_date.append(month_before)
    
fin_df['month_date'] = month_date

# 5. 한달 전 날짜에 맞춰 콜 금리 추가하기
month_rate=[]
for target_date in fin_df['month_date']:
    try:
        rate = fin_df._get_value(target_date, 'callrate') 
        month_rate.append(rate)
    except: # keyError 제외
        month_rate.append(0.0)
fin_df['month_rate'] = month_rate
fin_df

,Unnamed: 0,callrate,month_date,month_rate
date,,,,
2011-12-31,1,3.29,2011-11-30,0.00
2012-01-01,2,3.29,2011-12-01,0.00
2012-01-02,3,3.25,2011-12-02,0.00
2012-01-03,4,3.25,2011-12-03,0.00
2012-01-04,5,3.24,2011-12-04,0.00
...,...,...,...,...
2022-11-14,3972,3.07,2022-10-14,3.07
2022-11-15,3973,3.03,2022-10-15,3.07
2022-11-16,3974,3.03,2022-10-16,3.07


In [14]:
# 6. 콜 금리의 차이를 구하고 0 초과 : 'up', 0 같음 : 'same', 0 미만 : 'down'으로 라벨링
value_diff = fin_df['callrate'] - fin_df['month_rate']

fin_df['label']='0'
for i, diff in enumerate(value_diff):
    if diff > 0:
        fin_df['label'][i] = 'up'
    elif diff == 0:
        fin_df['label'][i] = 'same'
    elif diff < 0:
        fin_df['label'][i] = 'down'   

# 7. 2012년 ~ 2022년만 남기기
fin_df = fin_df.loc['2012-01-01':'2022-11-18', :]
fin_df.reset_index(inplace=True)

fin_df.to_csv('../data/callrate_preprocessing.csv',index=False)
fin_df

/var/folders/85/178rxd5d5b31szqc8b3mznd40000gn/T/ipykernel_1779/1618515152.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fin_df['label'][i] = 'up'
/var/folders/85/178rxd5d5b31szqc8b3mznd40000gn/T/ipykernel_1779/1618515152.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fin_df['label'][i] = 'down'
/var/folders/85/178rxd5d5b31szqc8b3mznd40000gn/T/ipykernel_1779/1618515152.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fin

,date,Unnamed: 0,callrate,month_date,month_rate,label
0,2012-01-01,2,3.29,2011-12-01,0.00,up
1,2012-01-02,3,3.25,2011-12-02,0.00,up
2,2012-01-03,4,3.25,2011-12-03,0.00,up
3,2012-01-04,5,3.24,2011-12-04,0.00,up
4,2012-01-05,6,3.25,2011-12-05,0.00,up
...,...,...,...,...,...,...
3970,2022-11-14,3972,3.07,2022-10-14,3.07,same
3971,2022-11-15,3973,3.03,2022-10-15,3.07,down
3972,2022-11-16,3974,3.03,2022-10-16,3.07,down
3973,2022-11-17,3975,3.02,2022-10-17,3.05,down
